<h1 style="border: thick double #32a1ce; text-align:center;border-radius:35px">
Projet Python pour la data science
</h1>

In [14]:
# Librairies standards
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from math import radians, cos, sin, asin, sqrt
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

<h1 style=" text-align:center">
1.Description des données 
</h1>

<h2 style=" text-align:left">
1.1- Paramètres 
</h2>

In [15]:
# Définition du répertoire de travail
HOME_DIR = Path.cwd().parent
DATA_DIR = Path(HOME_DIR, "data")
print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

Work directory: /home/onyxia/work/projet-Python/projet 
Data directory: /home/onyxia/work/projet-Python/projet/data


<h2 style=" text-align:left">
1.2- Chargement des bases
</h2>

Dans le cadre de notre travail nous allons utiliser 05 base bases de données qui porte essentiellement sur: 
   dans cette partie il est question de chargé et visualiser nos  differentes  bases de données.

<h3 style=" text-align:left">
a. Données sur la qualité de l'air  
</h3>

In [ ]:
qualite_air = pd.read_csv(Path(DATA_DIR, "indices_QA_commune_IDF_2017.csv"), sep=",")
qualite_air.head(5)

,date,ninsee,no2,o3,pm10
0,01/01/2017,0,23,16,43
1,01/01/2017,75,23,8,38
2,01/01/2017,77,19,17,44
3,01/01/2017,78,16,15,32
4,01/01/2017,91,14,16,34


In [22]:
qualite_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476485 entries, 0 to 476484
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    476485 non-null  object
 1   ninsee  476485 non-null  int64 
 2   no2     476485 non-null  int64 
 3   o3      476485 non-null  int64 
 4   pm10    476485 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 18.2+ MB


<u > Description </u> : Notre  base ne contient aucune valeur manquante. 

<u>Variable </u>: 
<ul>
    <li>date </li>
    <li>ninsee: code postale des département de l'ile de France</li>
    <li>no2: quantité de NO2 présente dans l'air à cette date</li>
    <li> o3: quantité de 03 présente dans l'air à cette date</li>
    <li> pm10: quantité de PM10 présente dans l'air à cette date</li>
</ul>



<h3 style=" text-align:left">
b. Données sur la Comorbidité 
</h3>

In [25]:
Comorbidite = pd.read_csv(Path(DATA_DIR, "Comorbidites.csv"))

FileNotFoundError: [Errno 2] No such file or directory: '/home/onyxia/work/projet-Python/projet/data/Comorbidites.csv'